In [1]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import matplotlib

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV

In [2]:
df = pd.read_csv('pred_crim_train.csv', delimiter=',')
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

Id - 0%
Expected - 0%
НЕВЕРОТН.1..Невероятные.приключения.итальянцев.в.России - 9%
ТЕЛОХРАН.2..Телохранитель - 15%
КАВКАЗСК.3..Кавказская.пленница.или.Новые.приключения.Шурика - 15%
ПТЫЙ_ЭЛЕ.4..Пятый.элемент - 9%
ТЕРМИНАТ.5..Терминатор - 17%
КРЕПКИЙ.6..Крепкий.орешек - 21%
ПОЛИЦ_АК.7..Полицейская.академия - 18%
ЧУЖИЕ.8..Чужие - 21%
КРОКОДИЛ.9..Крокодил.Данди - 26%
ЗДРАВСТВ.10..Здравствуйте.я.ваша.тетя. - 12%
ИРОНИ_СУ.11..Ирония.судьбы.или.с.легким.паром - 15%
КРАСОТКА.12..Красотка - 18%
МАТРИЦА.13..Матрица - 14%
ТИТАНИК.14..Титаник - 19%
К_9.15..К.9 - 40%
НАЗАД_В.16..Назад.в.будущее - 18%
САМА_ОБА.17..Сама.обательная.и.привлекательна - 22%
БЛИЗНЕЦЫ.18..Близнецы - 26%
СОБАЧЬЕ.19..Собачье.сердце - 11%
СОБАКА_Б.20..Собака.Баскервилей - 16%
МЕСТО_ВС.21..Место.встречи.изменить.нельзя - 13%
ГОРЕЦ.22..Горец - 27%
СЛУЖЕБНЫ.23..Служебный.роман - 20%
ОС_РЫБ.25..Особенности.национальной.рыбалки - 18%
ОС_ОХОТЫ.26..Особенности.национальной.охоты - 13%
ДЕСТЬ_НЕ.27..Десть.негритят - 20%
ТЕРМИНA2.28..

Оставим признаки, у которых пропусков меньше 2/3 от общего количества. Заполним пропуски медианным значением.

In [3]:
df = df.dropna(axis = 1, thresh=2*len(df)/3)
df = df.fillna(df.median())#interpolate(method='polynomial', order=2)
df = df.dropna()
X = df.drop(['Expected', 'Id'], axis=1)
y = df['Expected']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.25, random_state=42)

In [ ]:
scorer = make_scorer(mean_squared_error, greater_is_better = False)

def rmse_cv_train(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring = scorer, cv = 10))
    return(rmse)

def rmse_cv_test(model):
    rmse= np.sqrt(-cross_val_score(model, X_test, y_test, scoring = scorer, cv = 10))
    return(rmse)

Будем проверять 4 модели регрессии-- линейная регрессия, lasso, ridge и elasticnet для 3 вариантов-- без нормировки, с min-max нормировкой и с нормировкой на стандартное отклонение.

In [6]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print("RMSE на тренировочном множестве :", rmse_cv_train(lr).mean())
print("RMSE на тестовом множестве :", rmse_cv_test(lr).mean())

RMSE на тренировочном множестве : 2.0513823773152233
RMSE на тестовом множестве : 2.1236672182221126


In [7]:
ridge = RidgeCV(alphas = [1000, 1200, 1500])
ridge.fit(X_train, y_train)
alpha = ridge.alpha_
print("Лучшая lambda :", alpha)

print("Попробуем ещё раз с lambda центрированной вокруг " + str(alpha))
ridge = RidgeCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, alpha * .85, 
                          alpha * .9, alpha * .95, alpha, alpha * 1.05, alpha * 1.1, alpha * 1.15,
                          alpha * 1.25, alpha * 1.3, alpha * 1.35, alpha * 1.4], 
                cv = 10)
ridge.fit(X_train, y_train)
alpha = ridge.alpha_
print("Лучшая lambda :", alpha)

print("Ridge RMSE на тренировочном множества :", rmse_cv_train(ridge).mean())
print("Ridge RMSE на тестовом множестве :", rmse_cv_test(ridge).mean())

Лучшая lambda : 1200
Попробуем ещё раз с lambda центрированной вокруг 1200
Лучшая lambda : 1200.0
Ridge RMSE на тренировочном множества : 1.9992714657849988
Ridge RMSE на тестовом множестве : 1.8773675387629745


In [8]:
lasso = LassoCV(alphas = [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 
                          0.3, 0.6, 1], 
                max_iter = 50000, cv = 10)
lasso.fit(X_train, y_train)
alpha = lasso.alpha_
print("Лучшая lambda :", alpha)

print("Попробуем ещё раз с lambda центрированной вокруг " + str(alpha))
lasso = LassoCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, 
                          alpha * .85, alpha * .9, alpha * .95, alpha, alpha * 1.05, 
                          alpha * 1.1, alpha * 1.15, alpha * 1.25, alpha * 1.3, alpha * 1.35, 
                          alpha * 1.4], 
                max_iter = 50000, cv = 10)
lasso.fit(X_train, y_train)
alpha = lasso.alpha_
print("Лучшая lambda :", alpha)

print("Lasso RMSE на тренировочном множестве :", rmse_cv_train(lasso).mean())
print("Lasso RMSE на тестовом множестве :", rmse_cv_test(lasso).mean())

Лучшая lambda : 0.1
Попробуем ещё раз с lambda центрированной вокруг 0.1
Лучшая lambda : 0.13999999999999999
Lasso RMSE на тренировочном множестве : 1.9569135505691737
Lasso RMSE на тестовом множестве : 1.9204686806139866


In [9]:
elastic = ElasticNetCV(alphas = [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 
                          0.3, 0.6, 1], 
                max_iter = 50000, cv = 10)
elastic.fit(X_train, y_train)
alpha = elastic.alpha_
print("Лучшая lambda :", alpha)

print("Попробуем ещё раз с lambda центрированной вокруг " + str(alpha))
elastic = ElasticNetCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, 
                          alpha * .85, alpha * .9, alpha * .95, alpha, alpha * 1.05, 
                          alpha * 1.1, alpha * 1.15, alpha * 1.25, alpha * 1.3, alpha * 1.35, 
                          alpha * 1.4], 
                max_iter = 50000, cv = 10)
elastic.fit(X_train, y_train)
alpha = elastic.alpha_
print("Лучшая lambda :", alpha)

print("Lasso RMSE на тренировочном множестве :", rmse_cv_train(elastic).mean())
print("Lasso RMSE на тестовом множестве :", rmse_cv_test(elastic).mean())

Лучшая lambda : 0.3
Попробуем ещё раз с lambda центрированной вокруг 0.3
Лучшая lambda : 0.285
Lasso RMSE на тренировочном множестве : 1.9582479088636522
Lasso RMSE на тестовом множестве : 1.8969730164270906


In [10]:
lr.fit(X_train, y_train)
ridge.fit(X_train, y_train)
lasso.fit(X_train, y_train)
elastic.fit(X_train, y_train)

ElasticNetCV(alphas=[0.18, 0.195, 0.21, 0.22499999999999998, 0.24, 0.255, 0.27,
                     0.285, 0.3, 0.315, 0.33, 0.345, 0.375, 0.39, 0.405, 0.42],
             cv=10, max_iter=50000)

In [11]:
print(np.sqrt(mean_squared_error([12 if x>12 else x for x in lr.predict(X_test)], y_test)))
print(np.sqrt(mean_squared_error([12 if x>12 else x for x in ridge.predict(X_test)], y_test)))
print(np.sqrt(mean_squared_error([12 if x>12 else x for x in lasso.predict(X_test)], y_test)))
print(np.sqrt(mean_squared_error([12 if x>12 else x for x in elastic.predict(X_test)], y_test)))

1.8414254944358766
1.790123458168463
1.7921818998788972
1.7918944129757923


In [12]:
min_X = X_train.min()
max_X = X_train.max()
X_train_min_max = (X_train-min_X)/(max_X-min_X)
X_test_min_max = (X_test-min_X)/(max_X-min_X)

In [13]:
scorer = make_scorer(mean_squared_error, greater_is_better = False)

def rmse_cv_train(model):
    rmse= np.sqrt(-cross_val_score(model, X_train_min_max, y_train, scoring = scorer, cv = 10))
    return(rmse)

def rmse_cv_test(model):
    rmse= np.sqrt(-cross_val_score(model, X_test_min_max, y_test, scoring = scorer, cv = 10))
    return(rmse)

In [14]:
lr_min_max = LinearRegression()
lr_min_max.fit(X_train_min_max, y_train)
print("RMSE на тренировочном множестве :", rmse_cv_train(lr_min_max).mean())
print("RMSE на тестовом множестве :", rmse_cv_test(lr_min_max).mean())

RMSE на тренировочном множестве : 2.0513823773152233
RMSE на тестовом множестве : 2.123667218222113


In [15]:
ridge_min_max = RidgeCV(alphas = [8, 10, 12])
ridge_min_max.fit(X_train_min_max, y_train)
alpha = ridge_min_max.alpha_
print("Лучшая lambda :", alpha)

print("Попробуем ещё раз с lambda центрированной вокруг " + str(alpha))
ridge_min_max = RidgeCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, alpha * .85, 
                          alpha * .9, alpha * .95, alpha, alpha * 1.05, alpha * 1.1, alpha * 1.15,
                          alpha * 1.25, alpha * 1.3, alpha * 1.35, alpha * 1.4], 
                cv = 10)
ridge_min_max.fit(X_train_min_max, y_train)
alpha = ridge_min_max.alpha_
print("Лучшая lambda :", alpha)

print("Ridge RMSE на тренировочном множества :", rmse_cv_train(ridge_min_max).mean())
print("Ridge RMSE на тестовом множестве :", rmse_cv_test(ridge_min_max).mean())

Лучшая lambda : 10
Попробуем ещё раз с lambda центрированной вокруг 10
Лучшая lambda : 10.0
Ridge RMSE на тренировочном множества : 1.9982390208271181
Ridge RMSE на тестовом множестве : 1.8769601084521153


In [16]:
lasso_min_max = LassoCV(alphas = [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 
                          0.3, 0.6, 1], 
                max_iter = 50000, cv = 10)
lasso_min_max.fit(X_train_min_max, y_train)
alpha = lasso_min_max.alpha_
print("Лучшая lambda :", alpha)

print("Попробуем ещё раз с lambda центрированной вокруг " + str(alpha))
lasso_min_max = LassoCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, 
                          alpha * .85, alpha * .9, alpha * .95, alpha, alpha * 1.05, 
                          alpha * 1.1, alpha * 1.15, alpha * 1.25, alpha * 1.3, alpha * 1.35, 
                          alpha * 1.4], 
                max_iter = 50000, cv = 10)
lasso_min_max.fit(X_train_min_max, y_train)
alpha = lasso_min_max.alpha_
print("Лучшая lambda :", alpha)

print("Lasso RMSE на тренировочном множестве :", rmse_cv_train(lasso_min_max).mean())
print("Lasso RMSE на тестовом множестве :", rmse_cv_test(lasso_min_max).mean())

Лучшая lambda : 0.01
Попробуем ещё раз с lambda центрированной вокруг 0.01
Лучшая lambda : 0.013000000000000001
Lasso RMSE на тренировочном множестве : 1.9574220449568405
Lasso RMSE на тестовом множестве : 1.9164700820035148


In [17]:
elastic_min_max = ElasticNetCV(alphas = [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 
                          0.3, 0.6, 1], 
                max_iter = 50000, cv = 10)
elastic_min_max.fit(X_train_min_max, y_train)
alpha = elastic_min_max.alpha_
print("Лучшая lambda :", alpha)

print("Попробуем ещё раз с lambda центрированной вокруг " + str(alpha))
elastic_min_max = ElasticNetCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, 
                          alpha * .85, alpha * .9, alpha * .95, alpha, alpha * 1.05, 
                          alpha * 1.1, alpha * 1.15, alpha * 1.25, alpha * 1.3, alpha * 1.35, 
                          alpha * 1.4], 
                max_iter = 50000, cv = 10)
elastic_min_max.fit(X_train_min_max, y_train)
alpha = elastic_min_max.alpha_
print("Лучшая lambda :", alpha)

print("Lasso RMSE на тренировочном множестве :", rmse_cv_train(elastic_min_max).mean())
print("Lasso RMSE на тестовом множестве :", rmse_cv_test(elastic_min_max).mean())

Лучшая lambda : 0.01
Попробуем ещё раз с lambda центрированной вокруг 0.01
Лучшая lambda : 0.013999999999999999
Lasso RMSE на тренировочном множестве : 1.96482245557
Lasso RMSE на тестовом множестве : 1.8909870009887175


In [18]:
lr_min_max.fit(X_train_min_max, y_train)
ridge_min_max.fit(X_train_min_max, y_train)
lasso_min_max.fit(X_train_min_max, y_train)
elastic_min_max.fit(X_train_min_max, y_train)

ElasticNetCV(alphas=[0.006, 0.006500000000000001, 0.006999999999999999, 0.0075,
                     0.008, 0.0085, 0.009000000000000001, 0.0095, 0.01, 0.0105,
                     0.011000000000000001, 0.0115, 0.0125, 0.013000000000000001,
                     0.013500000000000002, 0.013999999999999999],
             cv=10, max_iter=50000)

In [19]:
print(np.sqrt(mean_squared_error([12 if x>12 else x for x in lr_min_max.predict(X_test_min_max)], y_test)))
print(np.sqrt(mean_squared_error([12 if x>12 else x for x in ridge_min_max.predict(X_test_min_max)], y_test)))
print(np.sqrt(mean_squared_error([12 if x>12 else x for x in lasso_min_max.predict(X_test_min_max)], y_test)))
print(np.sqrt(mean_squared_error([12 if x>12 else x for x in elastic_min_max.predict(X_test_min_max)], y_test)))

1.841425494435877
1.7901897690178474
1.7928887709177816
1.7858339165864467


In [20]:
mean_X = X_train.mean()
std_X = X_train.std()
X_train_std = (X_train-mean_X)/std_X
X_test_std = (X_test-mean_X)/std_X

In [21]:
scorer = make_scorer(mean_squared_error, greater_is_better = False)

def rmse_cv_train(model):
    rmse= np.sqrt(-cross_val_score(model, X_train_std, y_train, scoring = scorer, cv = 10))
    return(rmse)

def rmse_cv_test(model):
    rmse= np.sqrt(-cross_val_score(model, X_test_std, y_test, scoring = scorer, cv = 10))
    return(rmse)

In [22]:
lr_std = LinearRegression()
lr_std.fit(X_train_std, y_train)
print("RMSE на тренировочном множестве :", rmse_cv_train(lr_std).mean())
print("RMSE на тестовом множестве :", rmse_cv_test(lr_std).mean())

RMSE на тренировочном множестве : 2.0513823773152238
RMSE на тестовом множестве : 2.1236672182221126


In [23]:
ridge_std = RidgeCV(alphas = [280, 300, 320, 340])
ridge_std.fit(X_train_std, y_train)
alpha = ridge_std.alpha_
print("Лучшая lambda :", alpha)

print("Попробуем ещё раз с lambda центрированной вокруг " + str(alpha))
ridge_std = RidgeCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, alpha * .85, 
                          alpha * .9, alpha * .95, alpha, alpha * 1.05, alpha * 1.1, alpha * 1.15,
                          alpha * 1.25, alpha * 1.3, alpha * 1.35, alpha * 1.4], 
                cv = 10)
ridge_std.fit(X_train_std, y_train)
alpha = ridge_std.alpha_
print("Лучшая lambda :", alpha)

print("Ridge RMSE на тренировочном множества :", rmse_cv_train(ridge_std).mean())
print("Ridge RMSE на тестовом множестве :", rmse_cv_test(ridge_std).mean())

Лучшая lambda : 300
Попробуем ещё раз с lambda центрированной вокруг 300
Лучшая lambda : 300.0
Ridge RMSE на тренировочном множества : 2.0041585767167556
Ridge RMSE на тестовом множестве : 1.8930174829707132


In [24]:
lasso_std = LassoCV(alphas = [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 
                          0.3, 0.6, 1], 
                max_iter = 50000, cv = 10)
lasso_std.fit(X_train_std, y_train)
alpha = lasso_std.alpha_
print("Лучшая lambda :", alpha)

print("Попробуем ещё раз с lambda центрированной вокруг " + str(alpha))
lasso_std = LassoCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, 
                          alpha * .85, alpha * .9, alpha * .95, alpha, alpha * 1.05, 
                          alpha * 1.1, alpha * 1.15, alpha * 1.25, alpha * 1.3, alpha * 1.35, 
                          alpha * 1.4], 
                max_iter = 50000, cv = 10)
lasso_std.fit(X_train_std, y_train)
alpha = lasso.alpha_
print("Лучшая lambda :", alpha)

print("Lasso RMSE на тренировочном множестве :", rmse_cv_train(lasso_std).mean())
print("Lasso RMSE на тестовом множестве :", rmse_cv_test(lasso_std).mean())

Лучшая lambda : 0.1
Попробуем ещё раз с lambda центрированной вокруг 0.1
Лучшая lambda : 0.13999999999999999
Lasso RMSE на тренировочном множестве : 1.9601299414917173
Lasso RMSE на тестовом множестве : 1.9197941193095782


In [25]:
elastic_std = ElasticNetCV(alphas = [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 
                          0.3, 0.6, 1], 
                max_iter = 50000, cv = 10)
elastic_std.fit(X_train_std, y_train)
alpha = elastic_std.alpha_
print("Лучшая lambda :", alpha)

print("Попробуем ещё раз с lambda центрированной вокруг " + str(alpha))
elastic_std = ElasticNetCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, 
                          alpha * .85, alpha * .9, alpha * .95, alpha, alpha * 1.05, 
                          alpha * 1.1, alpha * 1.15, alpha * 1.25, alpha * 1.3, alpha * 1.35, 
                          alpha * 1.4], 
                max_iter = 50000, cv = 10)
elastic_std.fit(X_train_std, y_train)
alpha = elastic_std.alpha_
print("Лучшая lambda :", alpha)

print("Lasso RMSE на тренировочном множестве :", rmse_cv_train(elastic_std).mean())
print("Lasso RMSE на тестовом множестве :", rmse_cv_test(elastic_std).mean())

Лучшая lambda : 0.1
Попробуем ещё раз с lambda центрированной вокруг 0.1
Лучшая lambda : 0.13999999999999999
Lasso RMSE на тренировочном множестве : 1.9576120761394047
Lasso RMSE на тестовом множестве : 1.911012071006288


In [26]:
lr_std.fit(X_train_std, y_train)
ridge_std.fit(X_train_std, y_train)
lasso_std.fit(X_train_std, y_train)
elastic_std.fit(X_train_std, y_train)

ElasticNetCV(alphas=[0.06, 0.065, 0.06999999999999999, 0.07500000000000001,
                     0.08000000000000002, 0.085, 0.09000000000000001, 0.095,
                     0.1, 0.10500000000000001, 0.11000000000000001,
                     0.11499999999999999, 0.125, 0.13, 0.135,
                     0.13999999999999999],
             cv=10, max_iter=50000)

In [27]:
print(np.sqrt(mean_squared_error([12 if x>12 else x for x in lr_std.predict(X_test_std)], y_test)))
print(np.sqrt(mean_squared_error([12 if x>12 else x for x in ridge_std.predict(X_test_std)], y_test)))
print(np.sqrt(mean_squared_error([12 if x>12 else x for x in lasso_std.predict(X_test_std)], y_test)))
print(np.sqrt(mean_squared_error([12 if x>12 else x for x in elastic_std.predict(X_test_std)], y_test)))

1.8414254944358768
1.7947683184052263
1.8015627972511374
1.7970215680483737


Для предсказания будем использовать метод показвший лучший результат-- ElasticNet с min-max нормировкой.

In [38]:
df_kag = pd.read_csv('pred_crim_test.csv', delimiter=';')
df_kag = df_kag.loc[:, df.columns.drop('Expected')]
df_kag_NA = df_kag.fillna(df_kag.median())
f = open('reg_test.csv','w') 
f.write('Id,Predicted')
f.write('\n')
minX = X.min()
maxX = X.max()
X_ = (X-minX)/(maxX-minX)
elastic_min_max = ElasticNetCV(alphas = [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 
                          0.3, 0.6, 1], 
                max_iter = 50000, cv = 10)
elastic_min_max.fit(X_, y)
alpha = elastic_min_max.alpha_
print("Лучшая lambda :", alpha)

print("Попробуем ещё раз с lambda центрированной вокруг " + str(alpha))
elastic_min_max = ElasticNetCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, 
                          alpha * .85, alpha * .9, alpha * .95, alpha, alpha * 1.05, 
                          alpha * 1.1, alpha * 1.15, alpha * 1.25, alpha * 1.3, alpha * 1.35, 
                          alpha * 1.4], 
                max_iter = 50000, cv = 10)
elastic_min_max.fit(X_, y)
pred = elastic_min_max.predict((df_kag_NA.drop('Id', axis=1)-minX)/(maxX-minX))
for i in range(len(df_kag_NA['Id'])):
    f.write(str(df_kag_NA['Id'][i]))
    f.write(',')
    p = pred[i]
    if (p > 12):
        p = 12
    f.write(str(p))
    f.write('\n')

Лучшая lambda : 0.01
Попробуем ещё раз с lambda центрированной вокруг 0.01
